# Problem Set 2 - Part 1: Ambiguity and Inference

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the following instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).


### Pre-requisite knowledge

Understanding of `problem-set-1`:
- First order logic
- Lambda calculus
- Feature unification context free grammar

### Instructions

- Follow the instructions step-by-step.
- Run all cells before modifying them to see how the code works.
- The tasks will ask you to modify cells in this notebook.
- Modify the cell to provide your answer and run a test before submiting the file.

In [1]:
# This task requires NLTK and Jupyter Notebook (IPython package).

import nltk
from nltk.grammar import FeatureGrammar
from nltk.sem import cooper_storage as cs

from utils import display_latex, display_translation, display_tree, display, Markdown
from copy import deepcopy

read_expr = nltk.sem.Expression.fromstring

Run the following cell to import functions and syntax specific to this problem set:

In [2]:
from utils2 import sem_parser, evaluate_sentences, syntax, syntax_notv

### 1. Ambiguity in quantifiers [6 marks in total]

Follow the descriptions and instructions in sections **(1.1)**, **(1.2)** and **(1.3)** to learn about the code and the examples. Then answer the questions in section **(1.4)**.

#### 1.1. FCFG with a SEM feature

You should already be familar with FCFG with a SEM feature. 
Use the code below to parse the following sentences to their semantic representations:

1. every dog bites a bone
2. a man gives a bone to every dog
3. Russia gives Moscow to Napoleon

In [3]:
sentences = [
    'all dogs bite a bone',
    'a man gives a bone to every dog',
    'Russia gives Moscow to Napoleon',
]

# if you want to see the parse tree change `verbose` to True:
sents_reps = sem_parser(sentences, syntax, verbose=False)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    for semrep in semreps:
        display_translation(f"{i+1}. {sent}", semrep)

"1. all dogs bite a bone": $\forall\ x.(dog(x)\ \rightarrow\ \exists\ z_{1}.(bone(z_{1})\ \land\ bite(x,z_{1})))$

"2. a man gives a bone to every dog": $\exists\ x.(man(x)\ \land\ \forall\ z_{3}.(dog(z_{3})\ \rightarrow\ \exists\ z_{2}.(bone(z_{2})\ \land\ give(x,z_{2},z_{3}))))$

"3. Russia gives Moscow to Napoleon": $give(russia,moscow,napoleon)$

#### 1.2. Evaluate sentences in a model

We have a world model with entities and sets to represent their properties and relationships. 
Run the code below to evaluate the sentence representations from the previous section in the world model defined below.

In [4]:
# a world model:
entities = """
russia    => c1
moscow    => c2
napoleon  => m2
"""
unaries = """
dog  => {d1, d2, d3} 
man  => {m1, m2} 
bone => {bn1, bn2} 
"""
binaries = """
bite => {(d1, bn1), (d2, bn2), (d3, bn2)} 
"""
ternaries = """
give => {(c1, c2, m2), (m1, bn1, d1), (m1, bn2, d2), (m1, bn2, d3)} 
"""

# world is a collection of entities and relations:
world = entities + unaries + binaries + ternaries

# parse the sentences:
sents_reps = sem_parser(sentences, syntax, verbose=False)
# evaluate them:
sents_reps_vals = evaluate_sentences(sents_reps, world)

# print all readings
for i, (sent, semreps_vals) in enumerate(sents_reps_vals.items()):
    for semrep, val in semreps_vals.items():
        print(f"{i+1}. {sent}:")
        display_translation(val, semrep)

1. all dogs bite a bone:


"True": $\forall\ x.(dog(x)\ \rightarrow\ \exists\ z_{4}.(bone(z_{4})\ \land\ bite(x,z_{4})))$

2. a man gives a bone to every dog:


"True": $\exists\ x.(man(x)\ \land\ \forall\ z_{6}.(dog(z_{6})\ \rightarrow\ \exists\ z_{5}.(bone(z_{5})\ \land\ give(x,z_{5},z_{6}))))$

3. Russia gives Moscow to Napoleon:


"True": $give(russia,moscow,napoleon)$

#### 1.3. Exploiting syntactic ambiguity to represent semantic ambiguity

Ditransitive verbs (`DTV`) have two objects. Use the code below to inspect the lexical representation of `give(s)` in the grammar. When constructing a `VP`, we take the semantic representation of a di-trasitive verb `DTV`as a function and apply it on both direct and indirect objects: `vp = ?dtv(?obj,?pp)`. The lambda bound variables `Y` and `X` are substituted with `?obj` and `?pp`.

In [5]:
print(syntax.productions()[-1])
print(syntax.productions()[-5])

DTV[NUM='pl', SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>, TNS='pres'] -> 'give'
VP[NUM=?n, SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n, SEM=?v] NP[SEM=?obj] PP[SEM=?pp, +TO]


In the lecture we discussed that without a mechanism such as Cooper storage we would only get one scoping of quanitifers.

One trick to ensure a different scoping for `give(s)` is  to create a parallel syntactic rule that contains a different internal order of composition of arguments in the `SEM`feature as follows: 

For example, we can change the compositional function from:
$$\lambda Y\ X\ x.X(\lambda z.Y(\lambda y.give(x,y,z)))$$
to:
$$\lambda Y\ X\ x.Y(\lambda y.X(\lambda z.give(x,y,z)))$$

As we now have two rules for `DTV` this will introduce syntactic ambiguity when a parses encouters words such `give(s)`.
The following code adds two alternative rules to those in the previous cell. Run the code to see the parse results for sentences and their evaluation in the model.

In [6]:
fcfg_string_give = r"""
DTV[NUM=sg,SEM=<\Y X x.Y(\y.X(\z.give(x,y,z)))>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\Y X x.Y(\y.X(\z.give(x,y,z)))>,TNS=pres] -> 'give'
"""

# this is going to add new rules to the syntax:
new_syntax = FeatureGrammar(
    syntax.start(),
    syntax.productions() + FeatureGrammar.fromstring(fcfg_string_give).productions()
)

# parse the sentences with new syntax:
sents_reps = sem_parser(sentences, new_syntax, verbose=False)
# evaluate them:
sents_reps_vals = evaluate_sentences(sents_reps, world)

for i, (sent, semreps_vals) in enumerate(sents_reps_vals.items()):
    for semrep, val in semreps_vals.items():
        print(f"{i+1}. {sent}:")
        display_translation(val, semrep) 

1. all dogs bite a bone:


"True": $\forall\ x.(dog(x)\ \rightarrow\ \exists\ z_{7}.(bone(z_{7})\ \land\ bite(x,z_{7})))$

2. a man gives a bone to every dog:


"True": $\exists\ x.(man(x)\ \land\ \forall\ z_{11}.(dog(z_{11})\ \rightarrow\ \exists\ z_{10}.(bone(z_{10})\ \land\ give(x,z_{10},z_{11}))))$

2. a man gives a bone to every dog:


"False": $\exists\ x.(man(x)\ \land\ \exists\ z_{8}.(bone(z_{8})\ \land\ \forall\ z_{9}.(dog(z_{9})\ \rightarrow\ give(x,z_{8},z_{9}))))$

3. Russia gives Moscow to Napoleon:


"True": $give(russia,moscow,napoleon)$

#### 1.4. Questions

**1a.** Because the word `give`can now be expanded following two rules we get two extra parse trees for the second and and the third sentence. Answer the following questions: **[2 marks]**
   - Why sentence (2) with `give` has two different readings but sentence (3) has only one reading?
   - What is the difference between the two representations of sentence (2)?

**1b.** When you evalute sentence (2) in the model above, one reading is True and the other is False. Change the part of the model marked with `???` so that all readings all sentences will be True. **[1 marks]** 

Explain why the readings are true. **[+1 mark]**

In [7]:
ternaries = """
give => {(c1, c2, m2), (m1, bn1, d1), (m1, bn2, d2), (m1, bn2, d3), (m1, bn1, d1), (m1, bn1, d2), (m1, bn1, d3) }
"""

# new world
world = entities + unaries + binaries + ternaries

# evaluate them:
sents_reps_vals = evaluate_sentences(sents_reps, world)

for sent, semreps_vals in sents_reps_vals.items():
    for semrep, val in semreps_vals.items():
        print(f"{sent}:")
        display_translation(val, semrep)

all dogs bite a bone:


"True": $\forall\ x.(dog(x)\ \rightarrow\ \exists\ z_{7}.(bone(z_{7})\ \land\ bite(x,z_{7})))$

a man gives a bone to every dog:


"True": $\exists\ x.(man(x)\ \land\ \forall\ z_{11}.(dog(z_{11})\ \rightarrow\ \exists\ z_{10}.(bone(z_{10})\ \land\ give(x,z_{10},z_{11}))))$

a man gives a bone to every dog:


"True": $\exists\ x.(man(x)\ \land\ \exists\ z_{8}.(bone(z_{8})\ \land\ \forall\ z_{9}.(dog(z_{9})\ \rightarrow\ give(x,z_{8},z_{9}))))$

Russia gives Moscow to Napoleon:


"True": $give(russia,moscow,napoleon)$

**1c.** Consider a world with a `bite` relation as shown below.
Write *another* representation of the sentence "all dogs bite a bone" 
that is different from the one you get above and that is *True* in this model. 
Write your answer in `???`. **[1 mark]** 

In [8]:
binaries = """
bite => {(d1, bn1), (d2, bn1), (d3, bn1)} 
"""

# new world
world = entities + unaries + binaries + ternaries

sents_reps_copy = deepcopy(sents_reps)
sents_reps_copy['all dogs bite a bone'].append(
    read_expr(r"exists x.(bone(x) & all y.(dog(y) -> bite(y,x)))")
)

# evaluate them:
sents_reps_vals_copy = evaluate_sentences(sents_reps_copy, world)

for i, (sent, semreps_vals) in enumerate(sents_reps_vals_copy.items()):
    if sent == 'all dogs bite a bone':
        for semrep, val in semreps_vals.items():
            print(f"{i+1}. {sent}:")
            display_translation(val, semrep)

1. all dogs bite a bone:


"True": $\forall\ x.(dog(x)\ \rightarrow\ \exists\ z_{7}.(bone(z_{7})\ \land\ bite(x,z_{7})))$

1. all dogs bite a bone:


"True": $\exists\ x.(bone(x)\ \land\ \forall\ y.(dog(y)\ \rightarrow\ bite(y,x)))$

**1d.** Consider a new set of tuples representing the `give` predicate below.
Write *another* representation for "a man gives a bone to every dog"
that is *True* in the updated model.
Write your answer in `???`. **[1 mark]** 

In [15]:
ternaries = """
give => {(c1, c2, m2), (m2, bn1, d1), (m1, bn1, d2), (m1, bn1, d3)}
"""

# new world
world = entities + unaries + binaries + ternaries

sents_reps_copy = deepcopy(sents_reps)
sents_reps_copy['a man gives a bone to every dog'].append(
    read_expr(r"exists x.(man(x) & exists z.(bone(z) -> all y.(dog(y) -> give(x,y,z))))")
)

# evaluate them:
sents_reps_vals_copy = evaluate_sentences(sents_reps_copy, world)

for i, (sent, semreps_vals) in enumerate(sents_reps_vals_copy.items()):
    if sent == 'a man gives a bone to every dog':
        for semrep, val in semreps_vals.items():
            print(f"{i+1}. {sent}:")
            display_translation(val, semrep)

KeyError: 'a man gives a bone to every dog'

### 2. Cooper storage [8 marks in total]

#### 2.1. Learn about CooperSotrage
Study the following grammar with a `SEM` that is split between `CORE` and `STORE`. However, in this example, we never add anything to the `STORE`.

In [10]:
fcfg_storage_base = r"""
% start S

S[SEM=[CORE=<?subj(?vp)>, STORE=(?b1+?b2)]] -> NP[NUM=?n,SEM=[CORE=?subj, STORE=?b1]] VP[NUM=?n,SEM=[CORE=?vp, STORE=?b2]]

Nom[NUM=?n,SEM=?s] -> N[NUM=?n,SEM=?s]

VP[NUM=?n,SEM=?s] -> IV[NUM=?n,SEM=?s]
VP[NUM=?n,SEM=[CORE=<?v(?obj)>, STORE=(?b1+?b2)]] -> TV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]]
VP[NUM=?n,SEM=[CORE=<?v(?pp)(?obj)>, STORE=(?b1+?b2+?b3)]] -> DTV[NUM=?n,SEM=[CORE=?v, STORE=?b1]] NP[SEM=[CORE=?obj, STORE=?b2]] PP[+TO,SEM=[CORE=?pp, STORE=?b3]]

PP[+TO, SEM=[CORE=?np, STORE=?b1]] -> P[+TO] NP[SEM=[CORE=?np, STORE=?b1]]
"""

fcfg_storage_lexicon = r"""
PropN[NUM=sg,SEM=[CORE=<\P.P(angus)>, STORE=(/)]] -> 'Angus'
PropN[NUM=sg,SEM=[CORE=<\P.P(cyril)>, STORE=(/)]] -> 'Cyril'
PropN[NUM=sg,SEM=[CORE=<\P.P(irene)>, STORE=(/)]] -> 'Irene'

Det[NUM=sg,SEM=[CORE=<\P Q.all x.(P(x) -> Q(x))>, STORE=(/)]] -> 'every'
Det[NUM=sg,SEM=[CORE=<\P Q.exists x.(P(x) & Q(x))>, STORE=(/)]] -> 'a'

N[NUM=sg,SEM=[CORE=<\x.library(x)>, STORE=(/)]] -> 'library'
N[NUM=sg,SEM=[CORE=<\x.girl(x)>, STORE=(/)]] -> 'girl'
N[NUM=sg,SEM=[CORE=<\x.boy(x)>, STORE=(/)]] -> 'boy'
N[NUM=sg,SEM=[CORE=<\x.book(x)>, STORE=(/)]] -> 'book'

IV[NUM=sg,SEM=[CORE=<\x.smile(x)>, STORE=(/)],TNS=pres] -> 'smiles' 

TV[NUM=sg,SEM=[CORE=<\X x.X(\y.read(x,y))>, STORE=(/)],TNS=pres] -> 'reads'

DTV[NUM=sg,SEM=[CORE=<\Y X x.X(\z.Y(\y.give(x,y,z)))>, STORE=(/)],TNS=pres] -> 'gives'

P[+to] -> 'to'
""" 

fcfg_storage_np = r"""
NP[NUM=?n,SEM=[CORE=?np, STORE=?b1]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<?det(?nom)>, STORE=(?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""


sentences = [
    'Angus reads a book',
    'every girl reads a book',
    'every library gives a book to a girl',
]

fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. Angus reads a book


"1": $\exists\ z_{12}.(book(z_{12})\ \land\ read(angus,z_{12}))$

2. every girl reads a book


"1": $\forall\ x.(girl(x)\ \rightarrow\ \exists\ z_{13}.(book(z_{13})\ \land\ read(x,z_{13})))$

3. every library gives a book to a girl


"1": $\forall\ x.(library(x)\ \rightarrow\ \exists\ z_{15}.(book(z_{15})\ \land\ \exists\ z_{14}.(girl(z_{14})\ \land\ give(x,z_{14},z_{15}))))$

With the following change to the `NP` rule we push representations to the `STORE` and replace them with a simple varible in the `CORE` (which is then type raised to <<e,t>,t> so that it it can combine with a transitive verb entry).

In [11]:
fcfg_storage_np = r"""
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""

sentences = [
    'Angus reads a book',
    'every girl reads a book',
    'every library gives a book to a girl',
]

fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. Angus reads a book


"1": $\exists\ x.(book(x)\ \land\ read(angus,x))$

"2": $\exists\ x.(book(x)\ \land\ read(angus,x))$

2. every girl reads a book


"1": $\exists\ x.(book(x)\ \land\ \forall\ z_{16}.(girl(z_{16})\ \rightarrow\ read(z_{16},x)))$

...

"2": $\exists\ x.(book(x)\ \land\ \exists\ z_{21}.(girl(z_{21})\ \land\ \forall\ z_{20}.(library(z_{20})\ \rightarrow\ give(z_{20},z_{21},x))))$

"3": $\exists\ x.(girl(x)\ \land\ \forall\ z_{23}.(library(z_{23})\ \rightarrow\ \exists\ z_{22}.(book(z_{22})\ \land\ give(z_{23},x,z_{22}))))$

"4": $\forall\ x.(library(x)\ \rightarrow\ \exists\ z_{25}.(girl(z_{25})\ \land\ \exists\ z_{24}.(book(z_{24})\ \land\ give(x,z_{25},z_{24}))))$

"5": $\exists\ x.(book(x)\ \land\ \forall\ z_{27}.(library(z_{27})\ \rightarrow\ \exists\ z_{26}.(girl(z_{26})\ \land\ give(z_{27},z_{26},x))))$

"6": $\forall\ x.(library(x)\ \rightarrow\ \exists\ z_{29}.(book(z_{29})\ \land\ \exists\ z_{28}.(girl(z_{28})\ \land\ give(x,z_{28},z_{29}))))$

#### 2.2. Questions

**2a.** There are two identical readings of the first sentence. Why is this so? Suggest a change that would ensure you do not get doublicate readings for the first sentence but you get  alternative readings for the other sentence. **[2 marks]**

In [12]:
sentences = [
    'Angus reads a book',
    'every girl reads a book'
]

fcfg_storage_np = r"""
## Change the rule below:
#NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?np, @x)>+?b1)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=?b1]]
#Karolina+Tova: NP[NUM=?n,SEM=[CORE=?np, STORE=(/)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=(/)]]
NP[NUM=?n,SEM=[CORE=<?np>, STORE=(/)]] -> PropN[NUM=?n,SEM=[CORE=?np, STORE=(/)]]
NP[NUM=?n,SEM=[CORE=<\P.P(@x)>, STORE=(<bo(?det(?nom), @x)>+?b1+?b2)]] -> Det[NUM=?n,SEM=[CORE=?det, STORE=?b1]] Nom[NUM=?n,SEM=[CORE=?nom, STORE=?b2]]
"""


fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon 
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. Angus reads a book


"1": $\exists\ x.(book(x)\ \land\ read(angus,x))$

2. every girl reads a book


"1": $\exists\ x.(book(x)\ \land\ \forall\ z_{30}.(girl(z_{30})\ \rightarrow\ read(z_{30},x)))$

"2": $\forall\ x.(girl(x)\ \rightarrow\ \exists\ z_{31}.(book(z_{31})\ \land\ read(x,z_{31})))$

**2b.** Extend the grammar below to cover the following sentences: **[6 marks]**

In [16]:
sentences = [
    'every library gives a book to every girl and every boy',
    'no library gives every book to a boy',
    'a boy and a girl read all books',
    'Angus and Irene read all books',
]

# your answers
fcfg_storage_answers_1 = r"""
### Replace X with their proper representations
Det[NUM=sg,SEM=[CORE=<\P Q.(- exists x.(P(x) & Q(x)))>, STORE=(/)]] -> 'no'
Det[NUM=pl,SEM=[CORE=<\P \Q. all x.(P(x) -> Q(x))>, STORE=(/)]] -> 'all'
N[NUM=pl,SEM=[CORE=<\x.book(x)>, STORE=(/)]] -> 'books'
TV[NUM=pl,SEM=[CORE=<\X x.X(\y.read(x,y))>,STORE=(/)],TNS=pres] -> 'read'
"""

fcfg_storage_answers_2 = r"""
### Correct the conjunction rule here. Replace xxx and ??? with the correct answer:
NP[NUM=?n, SEM=[CORE=<\X.(?n1(X) & ?n2(X))>, STORE=(?b1+?b2)]] -> NP[NUM=?num1, SEM=[CORE=?n1, STORE=?b1]] CONJ NP[NUM=?num2, SEM=[CORE=?n2, STORE=?b2]]
CONJ -> 'and' 
"""

fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon + fcfg_storage_answers_1 + fcfg_storage_answers_2
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. every library gives a book to every girl and every boy


"1": $\forall\ x.(boy(x)\ \rightarrow\ \forall\ z_{562}.(girl(z_{562})\ \rightarrow\ \exists\ z_{561}.(book(z_{561})\ \land\ \forall\ z_{560}.(library(z_{560})\ \rightarrow\ (give(z_{560},z_{562},z_{561})\ \land\ give(z_{560},x,z_{561}))))))$

"2": $\forall\ x.(girl(x)\ \rightarrow\ \forall\ z_{565}.(boy(z_{565})\ \rightarrow\ \exists\ z_{564}.(book(z_{564})\ \land\ \forall\ z_{563}.(library(z_{563})\ \rightarrow\ (give(z_{563},x,z_{564})\ \land\ give(z_{563},z_{565},z_{564}))))))$

"3": $\forall\ x.(boy(x)\ \rightarrow\ \exists\ z_{568}.(book(z_{568})\ \land\ \forall\ z_{567}.(girl(z_{567})\ \rightarrow\ \forall\ z_{566}.(library(z_{566})\ \rightarrow\ (give(z_{566},z_{567},z_{568})\ \land\ give(z_{566},x,z_{568}))))))$

"4": $\exists\ x.(book(x)\ \land\ \forall\ z_{571}.(boy(z_{571})\ \rightarrow\ \forall\ z_{570}.(girl(z_{570})\ \rightarrow\ \forall\ z_{569}.(library(z_{569})\ \rightarrow\ (give(z_{569},z_{570},x)\ \land\ give(z_{569},z_{571},x))))))$

...

"4": $\exists\ x.(boy(x)\ \land\ \forall\ z_{651}.(book(z_{651})\ \rightarrow\ \exists\ z_{650}.(girl(z_{650})\ \land\ read(x,z_{651})\ \land\ read(z_{650},z_{651}))))$

"5": $\exists\ x.(girl(x)\ \land\ \exists\ z_{653}.(boy(z_{653})\ \land\ \forall\ z_{652}.(book(z_{652})\ \rightarrow\ (read(z_{653},z_{652})\ \land\ read(x,z_{652})))))$

"6": $\exists\ x.(boy(x)\ \land\ \exists\ z_{655}.(girl(z_{655})\ \land\ \forall\ z_{654}.(book(z_{654})\ \rightarrow\ (read(x,z_{654})\ \land\ read(z_{655},z_{654})))))$

4. Angus and Irene read all books


"1": $\forall\ x.(book(x)\ \rightarrow\ (read(angus,x)\ \land\ read(irene,x)))$

**2c.** Add the quantified expressions to the `STORE` only in the conjunction rule. Compare the number of readings you get now for the sentence below. Are there any invalid readings? If so, why? **[3 marks]**

In [14]:
sentences = [
    'every library gives a book to every girl and every boy',
]

fcfg_storage_answers_2 = r"""
### Correct the conjunction rule here. Replace xxx and ??? with the correct answer:
#NP[NUM=xxx, SEM=[CORE=<\P.P(@x)>, STORE=((<bo(???, @x)>)+?b1+?b2)]] -> NP[NUM=?num1, SEM=[CORE=?n1, STORE=?b1]] CONJ NP[NUM=?num2, SEM=[CORE=?n2, STORE=?b2]]
NP[NUM=?n, SEM=[CORE=<\P.P(@x)>, STORE=((<bo(\P.(?n1(P) & ?n2(P)), @x)>)+?b1+?b2)]] -> NP[NUM=?num1, SEM=[CORE=?n1, STORE=?b1]] CONJ NP[NUM=?num2, SEM=[CORE=?n2, STORE=?b2]]
CONJ -> 'and'  
"""


fcfg_storage = fcfg_storage_base + fcfg_storage_np + fcfg_storage_lexicon + fcfg_storage_answers_1 + fcfg_storage_answers_2
cs_syntax = FeatureGrammar.fromstring(fcfg_storage)
sents_reps = sem_parser(sentences, cs_syntax, verbose=False, is_cs=True)

for i, (sent, semreps) in enumerate(sents_reps.items()):
    counter = 0
    print(f"{i+1}. {sent}")
    for semrep in semreps:
        counter += 1
        display_translation(counter, semrep)

1. every library gives a book to every girl and every boy


"1": $\forall\ x.(boy(x)\ \rightarrow\ \forall\ z_{131}.(girl(z_{131})\ \rightarrow\ (\exists\ z_{129}.(book(z_{129})\ \land\ \forall\ z_{128}.(library(z_{128})\ \rightarrow\ give(z_{128},z_{131},z_{129})))\ \land\ \exists\ z_{130}.(book(z_{130})\ \land\ \forall\ z_{128}.(library(z_{128})\ \rightarrow\ give(z_{128},x,z_{130}))))))$

"2": $\forall\ x.(girl(x)\ \rightarrow\ \forall\ z_{135}.(boy(z_{135})\ \rightarrow\ (\exists\ z_{133}.(book(z_{133})\ \land\ \forall\ z_{132}.(library(z_{132})\ \rightarrow\ give(z_{132},x,z_{133})))\ \land\ \exists\ z_{134}.(book(z_{134})\ \land\ \forall\ z_{132}.(library(z_{132})\ \rightarrow\ give(z_{132},z_{135},z_{134}))))))$

"3": $\forall\ x.(boy(x)\ \rightarrow\ (\forall\ z_{138}.(girl(z_{138})\ \rightarrow\ \exists\ z_{137}.(book(z_{137})\ \land\ \forall\ z_{136}.(library(z_{136})\ \rightarrow\ give(z_{136},@x9,z_{137}))))\ \land\ \forall\ z_{139}.(girl(z_{139})\ \rightarrow\ \exists\ z_{137}.(book(z_{137})\ \land\ \forall\ z_{136}.(library(z_{136})\ \rightarrow\ give(z_{136},x,z_{137}))))))$

"4": $(\forall\ x.(boy(x)\ \rightarrow\ \forall\ z_{142}.(girl(z_{142})\ \rightarrow\ \exists\ z_{141}.(book(z_{141})\ \land\ \forall\ z_{140}.(library(z_{140})\ \rightarrow\ give(z_{140},@x9,z_{141})))))\ \land\ \forall\ x.(boy(x)\ \rightarrow\ \forall\ z_{142}.(girl(z_{142})\ \rightarrow\ \exists\ z_{141}.(book(z_{141})\ \land\ \forall\ z_{140}.(library(z_{140})\ \rightarrow\ give(z_{140},@x6,z_{141}))))))$

...

"116": $\exists\ x.(book(x)\ \land\ \forall\ z_{543}.(library(z_{543})\ \rightarrow\ \forall\ z_{542}.(girl(z_{542})\ \rightarrow\ \forall\ z_{541}.(boy(z_{541})\ \rightarrow\ give(z_{543},@x9,x))))\ \land\ \forall\ z_{544}.(library(z_{544})\ \rightarrow\ \forall\ z_{542}.(girl(z_{542})\ \rightarrow\ \forall\ z_{541}.(boy(z_{541})\ \rightarrow\ give(z_{544},@x6,x)))))$

"117": $(\forall\ x.(library(x)\ \rightarrow\ \exists\ z_{547}.(book(z_{547})\ \land\ \forall\ z_{546}.(girl(z_{546})\ \rightarrow\ \forall\ z_{545}.(boy(z_{545})\ \rightarrow\ give(x,@x9,z_{547})))))\ \land\ \forall\ x.(library(x)\ \rightarrow\ \exists\ z_{547}.(book(z_{547})\ \land\ \forall\ z_{546}.(girl(z_{546})\ \rightarrow\ \forall\ z_{545}.(boy(z_{545})\ \rightarrow\ give(x,@x6,z_{547}))))))$

"118": $\forall\ x.(library(x)\ \rightarrow\ (\exists\ z_{550}.(book(z_{550})\ \land\ \forall\ z_{549}.(girl(z_{549})\ \rightarrow\ \forall\ z_{548}.(boy(z_{548})\ \rightarrow\ give(x,@x9,z_{550}))))\ \land\ \exists\ z_{551}.(book(z_{551})\ \land\ \forall\ z_{549}.(girl(z_{549})\ \rightarrow\ \forall\ z_{548}.(boy(z_{548})\ \rightarrow\ give(x,@x6,z_{551}))))))$

"119": $\exists\ x.(book(x)\ \land\ \forall\ z_{555}.(library(z_{555})\ \rightarrow\ (\forall\ z_{553}.(girl(z_{553})\ \rightarrow\ \forall\ z_{552}.(boy(z_{552})\ \rightarrow\ give(z_{555},@x9,x)))\ \land\ \forall\ z_{554}.(girl(z_{554})\ \rightarrow\ \forall\ z_{552}.(boy(z_{552})\ \rightarrow\ give(z_{555},@x6,x))))))$

"120": $\forall\ x.(library(x)\ \rightarrow\ \exists\ z_{559}.(book(z_{559})\ \land\ \forall\ z_{557}.(girl(z_{557})\ \rightarrow\ \forall\ z_{556}.(boy(z_{556})\ \rightarrow\ give(x,@x9,z_{559})))\ \land\ \forall\ z_{558}.(girl(z_{558})\ \rightarrow\ \forall\ z_{556}.(boy(z_{556})\ \rightarrow\ give(x,@x6,z_{559})))))$

## Marks

This part of the assignment has a total of 14 marks.